In [576]:
library(mgcv)
library(dplyr)
library(gdata)
library(epitools)
library(tableone)

In [595]:
df<- read.csv("csv_folder/eicu_clean_19Aug.csv", stringsAsFactors = TRUE)
df$age<- as.integer(df$age)
df$gender<- factor(df$gender)
df$ethnicity<- factor(df$ethnicity)
cols <- c(18:29)
df[cols] <- lapply(df[cols], factor)  
df_eicu<- df[,-c(1,7,8,10,12,39,41)]
df_eicu$hosp_mort <-ifelse(df_eicu$hosp_mort=='ALIVE',0,1)
df_eicu$icu_mort<-ifelse(df_eicu$icu_mort=='ALIVE',0,1)
df_eicu$gender <-ifelse(df_eicu$gender=='Female','F','M')
#df_eicu$gender<-df_eicu$Gender
df_eicu <- df_eicu %>% rename(vasso_duration_24h = X24hrs_hr, 
                              vasso_duration_48h = X48hrs_hr,
                              vasso_duration_72h = X72hrs_hr,
                              mechanical_ventilation = ventilation_duration_days,
                              sepsis = sepsis_binary,
                              hospital_expire_flag = hosp_mort,
                              icu_mortality = icu_mort,
                              stroke = cebrovascular_accident,
                              afib = atrial_fibrillation
                                )
df_eicu$use_vasopressor<- ifelse(df_eicu$vasso_duration_24h>0,1,0)


df_eicu_aki <-df_eicu[df_eicu$mortality_7d == 'ALIVE',]
eicu_prop <- read.csv("csv_folder/eicu_prop_24h_test.csv",stringsAsFactors = TRUE)
df_eicu_prop <- merge(x = df_eicu, y = eicu_prop[ ,  -1], by = "patientunitstayid", all.x=TRUE)

head(df_eicu_prop, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,use_vasopressor,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,0,1.0000000,0.0000000,0.0000000,0.8750000,0.000000,0.1250000,1.0000000,0.000000000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,1,0.4579439,0.2523364,0.2897196,0.3457944,0.364486,0.2897196,0.9065421,0.009345794,0.08411215


In [608]:
df1 <- read.csv("csv_folder/mimic_clean_19Aug.csv", 
               stringsAsFactors = TRUE)
df1$age<- as.integer(df1$age)
df1$icu_mortality<- factor(df1$icu_mortality)

cols <- c(17:28,39,40 )
df1[cols] <- lapply(df1[cols], factor)  
df_mimic<- df1[,-c(1,7,8,10,12)]
df_mimic <- df_mimic %>% rename(vasso_duration_24h = first_24_vasso_hours, 
                                vasso_duration_48h = first_48_vasso_hours,
                                vasso_duration_72h = first_72_vasso_hours,
                                unittype = first_careunit,
                                ethnicity = ethnicity_new, 
                                AKI_7d =aki_7day,
                                sofatotal = sofa,
                                mechanical_ventilation = vent_duration_days,
                                chf = congestive_heart_failure,
                                stroke = cereb)
df_mimic$use_vasopressor <- ifelse(df_mimic$vasso_duration_24h>0,1,0)
# df_mimic$ethnicity <-df_mimic$ethnicity_new
# df_mimic$sofatotal <-df_mimic$sofa
# df_mimic$AKI_7d <-df_mimic$aki_7day
df_mimic_aki <-df_mimic[df_mimic$mortality_7d == 0,]
mimic_prop <- read.csv("csv_folder/mimic_prop_24h_test.csv",stringsAsFactors = TRUE)
mimic_prop_AKI <- read.csv("csv_folder/mimic_prop_map_48h_72h.csv",stringsAsFactors = TRUE)

df_mimic_prop <- merge(x = df_mimic, y = mimic_prop[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI <- merge(x = df_mimic, y = mimic_prop_AKI[ ,  -1], by = "icustay_id", all.x=TRUE)
head(df_mimic_prop, n = 2)

icustay_id,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,sofatotal,vasso_duration_24h,vasso_duration_48h,⋯,use_vasopressor,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
200003,48,M,WHITE,SICU,0,0,6,1.084444,17.33444,⋯,1,0.05882353,0.5882353,0.35294118,0.6875000,0.12500000,0.1875000,0.06250000,0.6250000,0.3125000
200009,47,F,WHITE,CSRU,0,0,3,0.000000,0.00000,⋯,0,0.64516129,0.2903226,0.06451613,0.7741935,0.06451613,0.1612903,0.03225806,0.3548387,0.6129032


# table 1 eicu

In [584]:

xvars<-c("gender","age","bmi","ethnicity","unittype","sofatotal",
         "apache_iv",
         "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h",
#          "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#          "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h",
    
         "hypertension","afib","cancer","chf","ckd","cld","copd","diabetes","ihd","stroke",
         "sepsis",
         
         "hospital_expire_flag","AKI_7d","icu_mortality","vasso_duration_24h","use_vasopressor","mechanical_ventilation"
        
        )
catvars<-c("gender","ethnicity","unittype","hypertension","afib","cancer","chf","ckd","cld","copd","diabetes","ihd","stroke",
         "sepsis","hospital_expire_flag","AKI_7d","icu_mortality","use_vasopressor","mechanical_ventilation")

In [667]:

TableOne_eICU<-CreateTableOne(var = xvars, factorVars =catvars,data=df_eicu)

print(TableOne_eICU, nonnormal = c("age","bmi","sofatotal","apache_iv",
                                    "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h","vasso_duration_24h"
#                                     "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#                                     "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h"
                                   ), 
      contDigits = 1, catDigits = 1, pDigits = 4, smd = TRUE,testName=TRUE)


                                   
                                    Overall             
  n                                 14374               
  gender = M (%)                     8439 (58.7)        
  age (median [IQR])                 65.0 [55.0, 74.0]  
  bmi (median [IQR])                 28.3 [24.2, 33.0]  
  ethnicity (%)                                         
     African American                1827 (12.7)        
     Asian                            134 ( 0.9)        
     Caucasian                      11218 (78.0)        
     Hispanic                         501 ( 3.5)        
     Native American                  101 ( 0.7)        
     Others                           593 ( 4.1)        
  unittype (%)                                          
     Cardiac ICU                      526 ( 3.7)        
     CCU-CTICU                       1508 (10.5)        
     CSICU                           1000 ( 7.0)        
     CTICU                           1596 (11.1)    

# table1 mimic

In [668]:

TableOne_eICU<-CreateTableOne(var = xvars, factorVars =catvars,data=df_mimic)

print(TableOne_eICU, nonnormal = c("age","bmi","sofatotal",
                                   #"apache_iv",
                                    "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h","vasso_duration_24h"
#                                     "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#                                     "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h"
                                   ), 
      contDigits = 1, catDigits = 1, pDigits = 4, smd = TRUE,testName=TRUE)


Warning message in ModuleReturnVarsExist(vars, data):
“The data frame does not have: apache_iv  Dropped”

                                   
                                    Overall             
  n                                 12291               
  gender = M (%)                     7228 (58.8)        
  age (median [IQR])                 66.0 [53.0, 76.0]  
  bmi (median [IQR])                 29.5 [25.7, 31.9]  
  ethnicity (%)                                         
     ASIAN                            291 ( 2.4)        
     BLACK                            735 ( 6.0)        
     HISPANIC                         366 ( 3.0)        
     OTHERS                          2354 (19.2)        
     WHITE                           8545 (69.5)        
  unittype (%)                                          
     CCU                             1290 (10.5)        
     CSRU                            3769 (30.7)        
     MICU                            2534 (20.6)        
     SICU                            2522 (20.5)        
     TSICU                           2176 (17.7)    

In [587]:
x<-df_eicu$vasso_duration_24h[df_eicu$vasso_duration_24h>0]
median(x)
quantile(x)

[1] 22.83

0%   25%   50%   75%  100% 
 0.02 17.43 22.83 23.95 24.00

In [200]:
# Model fitting
plot_width <- 17.2 * 0.393701 # The textwidth of our Word manuscript in inches.
#plot_width <- 4.2126 # The Lancet asks for plots to have a width of 107mm.
font <- "Times" # Text in figures should be Times New Roman according to the Lancet.
point_size <- 10 # Per Lancet instructions
logistic <- function(x) 1/(1+exp(-x))



# Function that adds labels to subfigures, edited from:
# https://logfc.wordpress.com/2017/03/15/adding-figure-labels-a-b-c-in-the-top-left-corner-of-the-plotting-region/
fig_label <- function(label) {
    cex <- 2
    ds <- dev.size("in")
    # xy coordinates of device corners in user coordinates
    x <- grconvertX(c(0, ds[1]), from="in", to="user")
    y <- grconvertY(c(0, ds[2]), from="in", to="user")
    
    # fragment of the device we use to plot
      # account for the fragment of the device that 
      # the figure is using
      fig <- par("fig")
      dx <- (x[2] - x[1])
      dy <- (y[2] - y[1])
      x <- x[1] + dx * fig[1:2]
      y <- y[1] + dy * fig[3:4]
  
  
  sw <- strwidth(label, cex=cex) * 60/100
  sh <- strheight(label, cex=cex) * 60/100
  
  x1 <- x[1] + sw
  
  y1 <- y[2] - sh
  
  old.par <- par(xpd=NA)
  on.exit(par(old.par))
  
  text(x1, y1, label, cex=cex)
}

In [510]:
gam_plotMed <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
  eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitMed$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2)
    lines(x = xx, y = ucl, lty = 2, lwd = 2)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
  }
  
  if(!missing(label)) fig_label(label)
}


In [511]:
find_gradient <- function(b)
{
xName <- colnames(b$model)[grep("ibp", colnames(b$model))]
eval(parse(text = paste(c('newdata = data.frame(',
                          xName, ' = b$model$', xName, ", gender = 'F', age = median(b$model$age), bmi = median(b$model$bmi),",
                  ifelse(
                    "high_vent_proportion" %in% colnames(b$model),
                    "high_vent_proportion = median(b$model$high_vent_proportion),",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"
                  ),
                  ifelse(
                    "ethnicity" %in% colnames(b$model),
                    "ethnicity = b$model$ethnicity,",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"

                  ),
                  ifelse(
                    "vasso_duration_24h" %in% colnames(b$model),
                    "vasso_duration_24h = median(b$model$vasso_duration_24h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_48h" %in% colnames(b$model),
                    "vasso_duration_48h = median(b$model$vasso_duration_48h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_72h" %in% colnames(b$model),
                    "vasso_duration_72h = median(b$model$vasso_duration_72h),",
                    ""
                  ),
                  "hospital_id = 264)"), collapse = "")))
# newd <- newdata
pred<-predict(b,newdata,type="link",se.fit = T)
eval(parse(text = paste0('xx <- b$model$', xName)))
# xx <- b$model$ibp_mean_24h
ord.index <- order(xx)
i<-2
j<-1
index <-1
bp_val <-list()
bp_grad<-list()
while (i <=length(ord.index)){
    eval(parse(text = paste(c('x2 <- b$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
    eval(parse(text = paste(c('x1 <- b$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]
    
    if((x2-x1)>=1.0){
        current_val <- pred$fit[ord.index[j]]
        right_val <- pred$fit[ord.index[i]]
        grad <-(right_val-current_val)/(x2-x1)
        bp_val[[index]] <- x2
        bp_grad[[index]]<-unlist(grad,use.names=FALSE)
        index <-index+1
        j <- i
        i <- i+1
        
      } else {
          i <-i+1
      }
      
    }
df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), "gradient" = unlist(bp_grad, use.names=FALSE))
return (df_gradient)
}

In [395]:
# find_gradient <- function(b)
# {
# xName <- colnames(b$model)[grep("ibp", colnames(b$model))]
# eval(parse(text = paste(c('newdata = data.frame(',
#                           xName, ' = b$model$', xName, ", gender = 'F', age = median(b$model$age), bmi = median(b$model$bmi),",
#                   ifelse(
#                     "high_vent_proportion" %in% colnames(b$model),
#                     "high_vent_proportion = median(b$model$high_vent_proportion),",
#                     ""
#                   ),
#                   ifelse(
#                     "apsiii" %in% colnames(b$model),
#                     "apsiii = median(b$model$apsiii),",
#                     "sofatotal = median(b$model$sofatotal),"
#                   ),
#                   ifelse(
#                     "vent_duration" %in% colnames(b$model),
#                     "vent_duration = median(b$model$vent_duration),",
#                     ""
#                   ),
#                   "hospital_id = 264)"), collapse = "")))
# # newd <- newdata
# pred<-predict(b,newdata,type="link",se.fit = T)
# eval(parse(text = paste0('xx <- b$model$', xName)))
# # xx <- b$model$ibp_mean_24h
# ord.index <- order(xx)
# i<-2
# j<-1
# index <-1
# bp_val <-list()
# bp_grad<-list()
# while (i <=length(ord.index)){
#     eval(parse(text = paste(c('x2 <- b$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
#     eval(parse(text = paste(c('x1 <- b$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
# #       x2 <- b$model$ibp_mean_24h[ord.index[i]]
# #       x1 <-b$model$ibp_mean_24h[ord.index[j]]
    
#     if((x2-x1)>=1.0){
#         current_val <- pred$fit[ord.index[j]]
#         right_val <- pred$fit[ord.index[i]]
#         grad <-(right_val-current_val)/(x2-x1)
#         bp_val[[index]] <- x2
#         bp_grad[[index]]<-unlist(grad,use.names=FALSE)
#         index <-index+1
#         j <- i
#         i <- i+1
        
#       } else {
#           i <-i+1
#       }
      
#     }
# df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), "gradient" = unlist(bp_grad, use.names=FALSE))
# return (df_gradient)
# }

## mean vs hospital mortality

In [397]:
gamfitMed_eICU_MAP_24h <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_24h <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_MAP_48h <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_48h <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_MAP_72h <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_72h <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [669]:
pdf(file = "figure/MAP_Hosp.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(55, 95)
gam_plotMed(gamfitMed_eICU_MAP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MAP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MAP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

# Find optimal range

In [413]:
print("eICU") #73-79
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_24h) #73-75
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_48h) #76-79
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_72h)#77-79
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
print("MIMIC") #72-75
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_24h)#72-74
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_48h)#73-75
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_72h) #74-75
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]


[1] "eICU"


[1] 73 74 75

[1] 76 77 78 79

[1] 77 78 79

[1] "MIMIC"


[1] 72 73 74

[1] 73 74 75

[1] 74 75

## systolic vs hospital mortality

In [406]:
gamfitMed_eICU_ASP_24h <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_24h <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ASP_48h <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_48h <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ASP_72h <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_72h <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [407]:
pdf(file = "figure/ASP_Hosp.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)

gam_plotMed(gamfitMed_eICU_ASP_24h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_eICU_ASP_48h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_eICU_ASP_72h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [416]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_24h) #112-120
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_48h) #116-124
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_72h)#118-125
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
print("MIMIC") #112-120
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_24h)#112-114
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_48h)#115-118
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_72h) #115-120
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]


[1] "eICU"


[1] 112 113 114 115 116 117 118 119 120

[1] 116 117 118 119 120 121 122 123 124

[1] 118 119 120 121 122 123 124 125

[1] "MIMIC"


[1] 112 113 114

[1] 115 116 117 118

[1] 115 116 117 118 119 120

## diastolic vs hospital mortality

In [589]:
gamfitMed_eICU_ADP_24h <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_24h <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ADP_48h <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_48h <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ADP_72h <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_72h <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [590]:
pdf(file = "figure/ADP_Hosp.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(35, 75)
gam_plotMed(gamfitMed_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [419]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_24h) #50-54
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_48h) #53-59
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_72h)#54-61
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
print("MIMIC") #41-59
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_24h)#41-52
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_48h)#43-54
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_72h) #47-59
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]


[1] "eICU"


[1] 50 51 52 53 54

[1] 53 54 55 56 57 58 59

[1] 54 55 56 57 58 59 60 61

[1] "MIMIC"


[1] 41 42 43 44 45 46 47 48 49 50 51 52

[1] 43 44 45 46 47 48 49 50 51 52 53 54

[1] 47 48 49 52 53 54 55 56 57 58 59

## mean, systolic, diastolic vs ICU mortality

In [420]:
gamfitMed_ICU_eICU_MAP_24h <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_24h <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_MAP_48h <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_48h <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_MAP_72h <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_72h <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [421]:
pdf(file = "figure/MAP_ICUmort.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(55, 95)
gam_plotMed(gamfitMed_ICU_eICU_MAP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_eICU_MAP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_eICU_MAP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [422]:
print("eICU") #73-80
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_24h) #73-75
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_48h) #77-79
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_72h)#78-80
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
print("MIMIC") #73-77
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_24h)#73-74
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_48h)#74-77
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_72h) #75-77
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]


[1] "eICU"


[1] 73 74 75

[1] 77 78 79

[1] 78 79 80

[1] "MIMIC"


[1] 73 74

[1] 74 75 76 77

[1] 75 76 77

In [423]:
gamfitMed_ICU_eICU_ASP_24h <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_24h <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ASP_48h <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_48h <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ASP_72h <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_72h <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [424]:
pdf(file = "figure/ASP_ICUmort.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)

gam_plotMed(gamfitMed_ICU_eICU_ASP_24h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_ICU_eICU_ASP_48h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_ICU_eICU_ASP_72h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [425]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_24h) #113-120
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_48h) #116-126
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_72h)#118-128
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
print("MIMIC") #112-120
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_24h)#113-115
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_48h)#116-119
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_72h) #116-122
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]


[1] "eICU"


[1] 113 114 115 116 117 118 119 120

[1] 116 117 118 119 120 121 122 123 124 125 126

[1] 118 119 120 121 122 123 124 125 126 127 128

[1] "MIMIC"


[1] 113 114 115

[1] 116 117 118 119

[1] 116 117 118 119 120 121 122

In [429]:
gamfitMed_ICU_eICU_ADP_24h <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_24h <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ADP_48h <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_48h <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ADP_72h <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_72h <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


In [430]:
pdf(file = "figure/ADP_ICUmort.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(35, 75)
gam_plotMed(gamfitMed_ICU_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_ICU_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_ICU_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [431]:
print("eICU") #49-65
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_24h) #49-54
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_48h) #54-61
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_72h)#56-65
eicu_grad[(eicu_grad$gradient > -0.01) & (eicu_grad$gradient< 0.01) 
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
print("MIMIC") #41-60
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_24h)#41-54
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_48h)#46-56
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_72h) #47-60
mimic_grad[(mimic_grad$gradient > -0.01) & (mimic_grad$gradient< 0.01)
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]


[1] "eICU"


[1] 49 50 51 52 53 54

[1] 54 55 56 57 58 59 60 61

[1] 56 57 58 59 60 61 62 63 64 65

[1] "MIMIC"


[1] 41 42 43 44 45 46 47 48 49 50 51 52 53 54

[1] 46 47 48 49 50 51 52 53 54 55 56

[1] 47 48 53 54 60

## mean, systolic, diastolic vs AKI (7day)

In [485]:
gamfitMed_AKI_eICU_MAP_24h <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_24h <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_MAP_48h <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_48h <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_MAP_72h <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_72h <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [486]:
pdf(file = "figure/MAP_AKI.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(50, 100)
gam_plotMed(gamfitMed_AKI_eICU_MAP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "A", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_AKI_eICU_MAP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "C", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_AKI_eICU_MAP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "E", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_AKI_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "B", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_AKI_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "D", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_AKI_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "F", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
dev.off()

[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"


png 
  2

In [497]:
print("eICU") # NA
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_24h) 
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_48h) 
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_72h)
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_24h)
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_48h)         #75
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_72h)         #76
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]


[1] "eICU"


,bp_value,gradient
29,71,-0.01115405
30,72,-0.01115359
31,73,-0.01115426
32,74,-0.01114986
33,75,-0.01113311
34,76,-0.01109595
35,77,-0.01102995
36,78,-0.01092676
37,79,-0.01077909


,bp_value,gradient
25,71,-0.01452056
26,72,-0.01448906
27,73,-0.01445711
28,74,-0.01442019
29,75,-0.01437254
30,76,-0.01430829
31,77,-0.01422036
32,78,-0.01409999
33,79,-0.01393912


,bp_value,gradient
24,71,-0.01227924
25,72,-0.01242600
26,73,-0.01258034
27,74,-0.01273590
28,75,-0.01288383
29,76,-0.01301545
30,77,-0.01312073
31,78,-0.01319011
32,79,-0.01321406


[1] "MIMIC"


,bp_value,gradient
30,71,-0.01721210
31,72,-0.01724483
32,73,-0.01720289
33,74,-0.01707690
34,75,-0.01685963
35,76,-0.01654661
36,77,-0.01613632
37,78,-0.01563024
38,79,-0.01503235


,bp_value,gradient
21,71,0.009279523
22,72,0.007217837
23,73,0.001502083
24,74,-0.006884152
25,75,-0.016794890
26,76,-0.027024948
27,77,-0.036497716
28,78,-0.044357918
29,79,-0.049827522


,bp_value,gradient
21,71,0.0035813657
22,72,0.0053105237
23,73,0.0037193957
24,74,-0.0007633375
25,75,-0.0075456787
26,76,-0.0158138980
27,77,-0.0246733409
28,78,-0.0332279289
29,79,-0.0406482636


In [499]:
gamfitMed_AKI_eICU_ASP_24h <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_24h <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ASP_48h <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_48h <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ASP_72h <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_72h <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [501]:
pdf(file = "figure/ASP_AKI.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
y_range = c(0.6, 1.0)
gam_plotMed(gamfitMed_AKI_eICU_ASP_24h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "A", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_AKI_eICU_ASP_48h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "C", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_AKI_eICU_ASP_72h, main = "eICU-CRD", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "E", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "B", xlabel ="Median of 24h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "D", xlabel ="Median of 48h Arterial Systolic Pressure (ASP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "F", xlabel ="Median of 72h Arterial Systolic Pressure (ASP)")
dev.off()

[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"


png 
  2

In [503]:
print("eICU") # NA
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_24h) 
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_48h) 
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_72h)
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_24h)
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 110),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_48h)         #111
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 100),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_72h)         # NA
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 100),]


[1] "eICU"


,bp_value,gradient
49,111,-0.004891041
50,112,-0.004891171
51,113,-0.004891283
52,114,-0.004891376
53,115,-0.004891446
54,116,-0.004891490
55,117,-0.004891508
56,118,-0.004891495
57,119,-0.004891450


,bp_value,gradient
45,111,-0.006006371
46,112,-0.006006418
47,113,-0.006006460
48,114,-0.006006497
49,115,-0.006006530
50,116,-0.006006558
51,117,-0.006006580
52,118,-0.006006597
53,119,-0.006006608


,bp_value,gradient
45,111,-0.004845360
46,112,-0.004845395
47,113,-0.004845430
48,114,-0.004845464
49,115,-0.004845499
50,116,-0.004845532
51,117,-0.004845564
52,118,-0.004845595
53,119,-0.004845624


[1] "MIMIC"


,bp_value,gradient
39,111,-0.01843080
40,112,-0.01831701
41,113,-0.01811814
42,114,-0.01783285
43,115,-0.01746174
44,116,-0.01700676
45,117,-0.01647114
46,118,-0.01585935
47,119,-0.01517702


,bp_value,gradient
24,101,-0.010935357
25,102,-0.008567701
26,103,-0.006555263
27,104,-0.004989815
28,105,-0.003966516
29,106,-0.003555489
30,107,-0.003793530
31,108,-0.004683394
32,109,-0.006202045
33,110,-0.008281559


,bp_value,gradient
23,101,-0.009786498
24,102,-0.008932456
25,103,-0.008268561
26,104,-0.007846578
27,105,-0.007708743
28,106,-0.007887767
29,107,-0.008400657
30,108,-0.009245803
31,109,-0.010402558
32,110,-0.011830978


In [504]:
gamfitMed_AKI_eICU_ADP_24h <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_24h <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ADP_48h <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_48h <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ADP_72h <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_72h <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [513]:
pdf(file = "figure/ADP_AKI.pdf", width = plot_width,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,2), cex = 1)
x_range = c(35, 75)
y_range = c(0.75,0.95)
y_range2 = c(0.6,1.0)
gam_plotMed(gamfitMed_AKI_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "A", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_AKI_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "C", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_AKI_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "E", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "B", xlabel ="Median of 24h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "D", xlabel ="Median of 48h Arterial Diastolic Pressure (ADP)")

gam_plotMed(gamfitMed_AKI_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "F", xlabel ="Median of 72h Arterial Diastolic Pressure (ADP)")
dev.off()

[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"


png 
  2

In [512]:
print("eICU") # NA
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_24h) 
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_48h) 
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_72h)
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_24h)
mimic_grad[(mimic_grad$bp_value< 60) & (mimic_grad$bp_value> 50),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_48h)         #58
mimic_grad[(mimic_grad$bp_value< 70) & (mimic_grad$bp_value> 50),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_72h)         #58
mimic_grad[(mimic_grad$bp_value< 70) & (mimic_grad$bp_value> 50),]


[1] "eICU"


,bp_value,gradient
30,51,-0.002591004
31,52,-0.002591108
32,53,-0.002591225
33,54,-0.002591357
34,55,-0.002591509
35,56,-0.002591682
36,57,-0.002591879
37,58,-0.002592104
38,59,-0.002592358


,bp_value,gradient
28,51,-0.003504361
29,52,-0.003504410
30,53,-0.003504462
31,54,-0.003504519
32,55,-0.003504582
33,56,-0.003504653
34,57,-0.003504733
35,58,-0.003504822
36,59,-0.003504921


,bp_value,gradient
27,51,-0.002467859
28,52,-0.002468090
29,53,-0.002468337
30,54,-0.002468605
31,55,-0.002468900
32,56,-0.002469224
33,57,-0.002469583
34,58,-0.002469978
35,59,-0.002470409


[1] "MIMIC"


,bp_value,gradient
27,51,-0.012078822
28,52,-0.011634887
29,53,-0.011186032
30,54,-0.010725570
31,55,-0.010246714
32,56,-0.009740759
33,57,-0.009198149
34,58,-0.008608164
35,59,-0.007962249


,bp_value,gradient
21,51,-1.879190e-04
22,52,9.908111e-05
23,53,-1.688913e-04
24,54,-8.702318e-04
25,55,-2.195920e-03
26,56,-4.434674e-03
27,57,-7.741094e-03
28,58,-1.211123e-02
29,59,-1.728093e-02
30,60,-2.262435e-02


,bp_value,gradient
20,51,-0.020836002
21,52,-0.014785716
22,53,-0.010038190
23,54,-0.006836564
24,55,-0.005313962
25,56,-0.005438286
26,57,-0.007043694
27,58,-0.009787966
28,59,-0.013189041
29,60,-0.016770994


# proportion plot and ORs

## use within_range_percent file to calculate the value and combine the csv at top

In [661]:
gam_plotProp <- function(
  gamfitProp,
  main = "Effect of treatment regime",
  yRange = c(0, .2),
  add = FALSE, col = "black",
  label,
  bp_type = "MAP",
  bp_low = 70,
  bp_high = 80
){
  print(paste("Number of cases:", summary(gamfitProp)$n))
  
  xName <- colnames(gamfitProp$model)[grep("prop", colnames(gamfitProp$model))]
  
  xRange = c(min(gamfitProp$model[,xName], na.rm= TRUE), max(gamfitProp$model[,xName], na.rm= TRUE))
#   xRange = c(0, max(gamfitProp$model[,xName], na.rm= TRUE))
  
  if(startsWith(xName, 'propBelow', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type," <", bp_low, "mmHg (black) or >",bp_high, "mmHg (blue)")
    #xlab <- expression("Proportion of  SpO"[2]*" measurements below 94%")
  } else if(startsWith(xName, 'propAbove', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type, " above ", bp_high, "mmHg")
  } else {
    xlab <- paste("Proportion of ",bp_type, " within ",bp_low, "mmHg to ", bp_high, "mmHg")
  }
  
  if(colnames(gamfitProp$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
      else if(colnames(gamfitProp$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  }
      else {
    ylab <- "Probability of ICU mortality"
  }
  
  if(!add) {
    plot(1, type = 'n', xlim = xRange, ylim = yRange,
           ylab = ylab,
           xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
      
      att <- pretty(yRange)
    if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
      axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
    } else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
      
      att <- pretty(xRange)
      axis(1, at = att, lab = paste0(att*100,'%'), las = TRUE)
  
  }
    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitProp, newdata = data.frame(', xName, 
                              ' = gamfitProp$model$', xName, 
                              ", gender = 'F', age = median(gamfitProp$model$age), bmi = median(gamfitProp$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitProp$model),
                        "high_vent_proportion = median(gamfitProp$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitProp$model),
                        "ethnicity = gamfitProp$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitProp$model),
                        "apsiii = median(gamfitProp$model$apsiii),",
                        "sofatotal = median(gamfitProp$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitProp$model),
                        "vasso_duration_24h = median(gamfitProp$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitProp$model),
                        "vasso_duration_48h = median(gamfitProp$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitProp$model),
                        "vasso_duration_72h = median(gamfitProp$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))
    
    
  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors 
  # for 'new' input
  eval(parse(text = paste0('xx <- gamfitProp$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitProp$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3, col = col)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3, col = col)
  }
  
  if(!missing(label)) fig_label(label)
}

In [592]:
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)

In [600]:
pdf(file = "figure/prop_MAP24_hosp.pdf", width = 8,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,2), cex = 1)
#summary(gamfitProp_eICU)
gam_plotProp(gamfitProp_eICU, main = "eICU", label = "A",bp_type = "MAP",bp_low = 73, bp_high = 79)
#summary(gamfitProp_below_eICU)
gam_plotProp(gamfitProp_below_eICU, main = "", label = "C",bp_type = "MAP",bp_low = 73, bp_high = 79)
#summary(gamfitProp_above_eICU)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "blue", add = TRUE)
#summary(gamfitProp_MIMIC)
gam_plotProp(gamfitProp_MIMIC, main = "MIMIC", label = "B", bp_type = "MAP", bp_low = 72, bp_high = 79)
#summary(gamfitProp_below_MIMIC)
gam_plotProp(gamfitProp_below_MIMIC, main = "", label = "D",bp_type = "MAP", bp_low = 72, bp_high = 79)
#summary(gamfitProp_above_MIMIC)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "blue", add = TRUE)
dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [662]:
gammfitProp_eICU <- gam(icu_mortality ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU <- gam(icu_mortality ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU <- gam(icu_mortality ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC <- gam(icu_mortality ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC <- gam(icu_mortality ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC <- gam(icu_mortality ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)

In [663]:
pdf(file = "figure/prop_MAP24_ICUmort.pdf", width = 8,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,2), cex = 1)
#summary(gamfitProp_eICU)
gam_plotProp(gamfitProp_eICU, main = "eICU", label = "A",bp_type = "MAP",bp_low = 73, bp_high = 79)
#summary(gamfitProp_below_eICU)
gam_plotProp(gamfitProp_below_eICU, main = "", label = "C",bp_type = "MAP",bp_low = 73, bp_high = 79)
#summary(gamfitProp_above_eICU)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "blue", add = TRUE)
#summary(gamfitProp_MIMIC)
gam_plotProp(gamfitProp_MIMIC, main = "MIMIC", label = "B", bp_type = "MAP", bp_low = 72, bp_high = 79)
#summary(gamfitProp_below_MIMIC)
gam_plotProp(gamfitProp_below_MIMIC, main = "", label = "D",bp_type = "MAP", bp_low = 72, bp_high = 79)
#summary(gamfitProp_above_MIMIC)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "blue", add = TRUE)
dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

## AKI prop for MAP48h and MAP72h

In [624]:
gam_plotProp_AKI <- function(
  gamfitProp,
  main = "Effect of treatment regime",
  yRange = c(0.6, 0.9),
  add = FALSE, col = "black",
  label,
  bp_type = "MAP",
  bp_low = 70
){
  print(paste("Number of cases:", summary(gamfitProp)$n))
  
  xName <- colnames(gamfitProp$model)[grep("prop", colnames(gamfitProp$model))]
  
  xRange = c(min(gamfitProp$model[,xName], na.rm= TRUE), max(gamfitProp$model[,xName], na.rm= TRUE))
#   xRange = c(0, max(gamfitProp$model[,xName], na.rm= TRUE))
  
  if(startsWith(xName, 'propBelow', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type," <", bp_low, "mmHg")
    #xlab <- expression("Proportion of  SpO"[2]*" measurements below 94%")
  }  else {
    xlab <- paste("Proportion of ",bp_type, " >= ",bp_low, "mmHg")
  }
  
  if(colnames(gamfitProp$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
      else if(colnames(gamfitProp$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  }
      else {
    ylab <- "Probability of ICU mortality"
  }
  
  if(!add) {
    plot(1, type = 'n', xlim = xRange, ylim = yRange,
           ylab = ylab,
           xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
      
      att <- pretty(yRange)
    if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
      axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
    } else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
      
      att <- pretty(xRange)
      axis(1, at = att, lab = paste0(att*100,'%'), las = TRUE)
  
  }
    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitProp, newdata = data.frame(', xName, 
                              ' = gamfitProp$model$', xName, 
                              ", gender = 'F', age = median(gamfitProp$model$age), bmi = median(gamfitProp$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitProp$model),
                        "high_vent_proportion = median(gamfitProp$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitProp$model),
                        "ethnicity = gamfitProp$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitProp$model),
                        "apsiii = median(gamfitProp$model$apsiii),",
                        "sofatotal = median(gamfitProp$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitProp$model),
                        "vasso_duration_24h = median(gamfitProp$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitProp$model),
                        "vasso_duration_48h = median(gamfitProp$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitProp$model),
                        "vasso_duration_72h = median(gamfitProp$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))
    
    
  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors 
  # for 'new' input
  eval(parse(text = paste0('xx <- gamfitProp$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitProp$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3, col = col)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3, col = col)
  }
  
  if(!missing(label)) fig_label(label)
}

In [622]:
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_map_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_prop_AKI, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_map_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_prop_AKI, family = binomial)


In [630]:
pdf(file = "figure/prop_MAP48_AKI.pdf", width = 4,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,1), cex = 1)

#summary(gamfitProp_MIMIC)
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 48h", label = "A", bp_type = "MAP", bp_low = 75)
#summary(gamfitProp_below_MIMIC)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 48h", label = "B",bp_type = "MAP", bp_low = 75)
dev.off()


[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [627]:
gamfitProp_MIMIC_72h_AKI <- gam(AKI_7d ~ s(prop_map_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_prop_AKI, family = binomial)
gamfitProp_below_MIMIC_72h_AKI <- gam(AKI_7d ~ s(propBelow_map_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_prop_AKI, family = binomial)

[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [629]:
pdf(file = "figure/prop_MAP72_AKI.pdf", width = 4,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,1), cex = 1)

#summary(gamfitProp_MIMIC)
gam_plotProp_AKI(gamfitProp_MIMIC_72h_AKI, main = "MIMIC 72h", label = "A", bp_type = "MAP", bp_low = 76)
#summary(gamfitProp_below_MIMIC)
gam_plotProp_AKI(gamfitProp_below_MIMIC_72h_AKI, main = "MIMIC 72h", label = "B",bp_type = "MAP", bp_low = 76)
dev.off()

[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [206]:

# df_mimic_compelete <- df_mimic_prop[complete.cases(df_mimic_prop[ , c('prop_asp_24','propBelow_asp_24','propAbove_asp_24')]),]

In [207]:
# df_mimic_subset_below<-df_mimic_prop[df_mimic_prop$prop_asp_24<=0.25,]
# df_mimic_subset_within <-df_mimic_prop
# df_mimic_subset_above<-df_mimic_prop[df_mimic_prop$prop_asp_24<=0.25,]

# dim(df_mimic_subset_below)[1]
# dim(df_mimic_subset_within)[1]
# dim(df_mimic_subset_above)[1]
# sum(dim(df_mimic_subset_below)[1], dim(df_mimic_subset_within)[1],dim(df_mimic_subset_above)[1])
# min(df_mimic_subset_below[,"propBelow_asp_24"])
# min(df_mimic_prop[df_mimic_prop$prop_asp_24<=0.4,44 ])

In [208]:
# head(df_mimic_prop[df_mimic_prop$prop_asp_24<=0.4,c(1,24,44 )],n =10)

In [209]:
# max(df_mimic_subset_above[,"propAbove_asp_24"],na.rm=T)

In [605]:
gammfitProp_eICU_asp <- gam(hospital_expire_flag ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_asp <- gam(hospital_expire_flag ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_asp <- gam(hospital_expire_flag ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_asp <- gam(hospital_expire_flag ~ s(prop_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_asp <- gam(hospital_expire_flag ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_asp <- gam(hospital_expire_flag ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)

In [606]:
pdf(file = "figure/prop_ASP24_hosp.pdf", width = 8,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,2), cex = 1)
#summary(gamfitProp_eICU)
gam_plotProp(gammfitProp_eICU_asp, main = "eICU", label = "A",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_below_eICU)
gam_plotProp(gamfitProp_below_eICU_asp, main = "", label = "C",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_above_eICU)
gam_plotProp(gamfitProp_above_eICU_asp, main = "", col = "blue", add = TRUE)
#summary(gamfitProp_MIMIC)
gam_plotProp(gamfitProp_MIMIC_asp, main = "MIMIC", label = "B", bp_type = "ASP", bp_low = 112, bp_high = 125)
#summary(gamfitProp_below_MIMIC)
gam_plotProp(gamfitProp_below_MIMIC_asp, main = "", label = "D",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_above_MIMIC)
gam_plotProp(gamfitProp_above_MIMIC_asp, main = "", col = "blue", add = TRUE)
dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [665]:
gammfitProp_eICU_asp <- gam(icu_mortality ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_asp <- gam(icu_mortality ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_asp <- gam(icu_mortality ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_asp <- gam(icu_mortality ~ s(prop_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_asp <- gam(icu_mortality ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_asp <- gam(icu_mortality ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)

In [666]:
pdf(file = "figure/prop_ASP24_ICUmort.pdf", width = 8,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,2), cex = 1)
#summary(gamfitProp_eICU)
gam_plotProp(gammfitProp_eICU_asp, main = "eICU", label = "A",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_below_eICU)
gam_plotProp(gamfitProp_below_eICU_asp, main = "", label = "C",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_above_eICU)
gam_plotProp(gamfitProp_above_eICU_asp, main = "", col = "blue", add = TRUE)
#summary(gamfitProp_MIMIC)
gam_plotProp(gamfitProp_MIMIC_asp, main = "MIMIC", label = "B", bp_type = "ASP", bp_low = 112, bp_high = 125)
#summary(gamfitProp_below_MIMIC)
gam_plotProp(gamfitProp_below_MIMIC_asp, main = "", label = "D",bp_type = "ASP",bp_low = 112, bp_high = 125)
#summary(gamfitProp_above_MIMIC)
gam_plotProp(gamfitProp_above_MIMIC_asp, main = "", col = "blue", add = TRUE)
dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [521]:
gammfitProp_eICU_adp      <- gam(hospital_expire_flag ~ s(prop_adp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_adp <- gam(hospital_expire_flag ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_adp <- gam(hospital_expire_flag ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_adp      <- gam(hospital_expire_flag ~ s(prop_adp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_adp <- gam(hospital_expire_flag ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_adp <- gam(hospital_expire_flag ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+(sofatotal)+s(vasso_duration_24h), data = df_mimic_prop, family = binomial)

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

In [523]:
pdf(file = "figure/prop_ADP24_hosp.pdf", width = 8,
    height = 6, pointsize = point_size, family = font)
par(mfcol = c(2,2), cex = 1)
#summary(gamfitProp_eICU)
gam_plotProp(gammfitProp_eICU_adp, main = "eICU-CRD", label = "A",bp_type = "ADP",bp_low = 50, bp_high = 54)
#summary(gamfitProp_below_eICU)
gam_plotProp(gamfitProp_below_eICU_adp, main = "", label = "C",bp_type = "ADP",bp_low = 50, bp_high = 54)
#summary(gamfitProp_above_eICU)
gam_plotProp(gamfitProp_above_eICU_adp, main = "", col = "blue", add = TRUE)
#summary(gamfitProp_MIMIC)
gam_plotProp(gamfitProp_MIMIC_adp, main = "MIMIC", label = "B", bp_type = "ADP", bp_low = 41, bp_high = 52)
#summary(gamfitProp_below_MIMIC)
gam_plotProp(gamfitProp_below_MIMIC_adp, main = "", label = "D",bp_type = "ADP",bp_low = 41, bp_high = 52)
#summary(gamfitProp_above_MIMIC)
gam_plotProp(gamfitProp_above_MIMIC_adp, main = "", col = "blue", add = TRUE)
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"


png 
  2

# Hospital/ICU Mortality odds ratio for MBP 5-15% vs 35-45% in-range

In [530]:
library(epitools)

In [551]:
head(df_eicu_prop, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,mortality_7d,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,ALIVE,1.0000000,0.0000000,0.0000000,0.8750000,0.000000,0.1250000,1.0000000,0.000000000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,ALIVE,0.4579439,0.2523364,0.2897196,0.3457944,0.364486,0.2897196,0.9065421,0.009345794,0.08411215


In [647]:
df_10<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.05 & df_eicu_prop$prop_map_24<=0.15,]
df_40<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.35 & df_eicu_prop$prop_map_24<=0.45,]

In [651]:
df_10<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.05 & df_eicu_prop$prop_asp_24<=0.15,]
df_40<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.35 & df_eicu_prop$prop_asp_24<=0.45,]

In [556]:
# df_10<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.05 & df_eicu_prop$prop_adp_24<=0.15,]
# df_40<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.35 & df_eicu_prop$prop_adp_24<=0.45,]

In [649]:
df_10<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.05 & df_mimic_prop$prop_map_24<=0.15,]
df_40<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.35 & df_mimic_prop$prop_map_24<=0.45,]

In [653]:
df_10<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.05 & df_mimic_prop$prop_asp_24<=0.15,]
df_40<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.35 & df_mimic_prop$prop_asp_24<=0.45,]

In [654]:
a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
c<-length(df_10[df_10$hospital_expire_flag==0,1])
b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
d<-length(df_40[df_40$hospital_expire_flag==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

[1]  444  164 1977 1323


[1] 0.1833953

[1] 0.1102892

,Disease1,Disease2,Total
Exposed1,444,1977,2421
Exposed2,164,1323,1487
Total,608,3300,3908
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.810469,1.496854,2.199155
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,4.482943e-10,5.44401e-10,9.255699e-10


In [638]:
a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
c<-length(df_10[df_10$icu_mortality==0,1])
b<-length(df_40[df_40$icu_mortality==1,1]) # control
d<-length(df_40[df_40$icu_mortality==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

[1]  361  131 2060 1356


[1] 0.1491119

[1] 0.08809684

,Disease1,Disease2,Total
Exposed1,361,2060,2421
Exposed2,131,1356,1487
Total,492,3416,3908
,estimate,lower,upper
Exposed1,1.0000,NA,NA
Exposed2,1.8124,1.470807,2.245677
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,1.23703e-08,1.333714e-08,2.371337e-08


# AKI ORs and CI

In [640]:
head(df_mimic_prop_AKI, n = 2)

icustay_id,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,sofatotal,vasso_duration_24h,vasso_duration_48h,⋯,ibp_mean_48h,ibp_mean_72h,bmi,AKI_7d,mortality_7d,use_vasopressor,prop_map_48_AKI,propBelow_map_48_AKI,prop_map_72_AKI,propBelow_map_72_AKI
200003,48,M,WHITE,SICU,0,0,6,1.084444,17.33444,⋯,77,78,25.06564,0,0,1,0.6078431,0.3921569,0.6753247,0.3246753
200009,47,F,WHITE,CSRU,0,0,3,0.000000,0.00000,⋯,76,76,34.05399,1,0,0,0.5818182,0.4181818,0.5272727,0.4727273


In [655]:
df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_48_AKI>=0.05 & df_mimic_prop_AKI$prop_map_48_AKI<=0.15,]
df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_48_AKI>=0.35 & df_mimic_prop_AKI$prop_map_48_AKI<=0.45,]

In [659]:
df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_72_AKI>=0.05 & df_mimic_prop_AKI$prop_map_72_AKI<=0.15,]
df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_72_AKI>=0.35 & df_mimic_prop_AKI$prop_map_72_AKI<=0.45,]

In [658]:
dim(df_10)
dim(df_40)

[1] 657  40

[1] 1263   40

In [660]:
a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
c<-length(df_10[df_10$AKI_7d==0,1])
b<-length(df_40[df_40$AKI_7d==1,1]) # control
d<-length(df_40[df_40$AKI_7d==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

[1]  599 1085   58  178


[1] 0.9117199

[1] 0.8590657

,Disease1,Disease2,Total
Exposed1,599,58,657
Exposed2,1085,178,1263
Total,1684,236,1920
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.690953,1.244049,2.329331
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.0006800098,0.0007387379,0.0008566447
